In [1]:
import numpy as np
import sys, os
sys.path.append(os.pardir + '/deep-learning-from-scratch/') 
from dataset.mnist import load_mnist
import pickle 
from common.util import im2col, col2im
from common.layers import Relu, SoftmaxWithLoss, Affine
import matplotlib.pyplot as plt
# from common.functions import *
# from common.gradient import numerical_gradient
# from tqdm import tqdm
from collections import OrderedDict
from common.trainer import Trainer

- im2colの引数
    - input_data：（データ数、チャンネル、高さ、横幅）
    - filter_h  ：フィルターの高さ
    - filter_w  ：フィルターの横幅
    - stride　　 : ストライド
    - pad   　　 : パディング

In [2]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        # reshapeの引数に-1を定義することで、多次元配列の要素数の辻褄が合うように要素数をまとめてくれる
        col_W = self.W.reshape(FN, -1).T
        out = np.dot(col, col_W) + self.b
        # transposeは多次元配列の軸の順序を変更させる
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        self.x = x
        self.col = col
        self.col_W = col_W

        return out

    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx

class Pooling:
        def __init__(self, pool_h, pool_w, stride = 2, pad = 0):
             self.pool_h = pool_h
             self.pool_w = pool_w
             self.stride = stride
             self.pad = pad
             self.x = None
             self.arg_max = None
        
        def forward(self, x):
             N, C, H, W = x.shape
             out_h = int(1 + (H - self.pool_h) / self.stride)
             out_w = int(1 + (W - self.pool_w) / self.stride)

             col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
             col = col.reshape(-1, self.pool_h*self.pool_w)

             arg_max = np.argmax(col, axis=1)
             out = np.max(col, axis=1)
             out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

             self.x = x
             self.arg_max = arg_max

             return out

        def backward(self, dout):
            dout = dout.transpose(0, 2, 3, 1)
            
            pool_size = self.pool_h * self.pool_w
            dmax = np.zeros((dout.size, pool_size))
            dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
            dmax = dmax.reshape(dout.shape + (pool_size,)) 
            
            dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
            dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
            
            return dx

class SimpleConvNet():
    def __init__(self, 
                  input_dim = (1, 28, 28), 
                  conv_param = {'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                  hidden_size = 100, output_size = 10, weight_init_std = 0.01):
          
          filter_num = conv_param['filter_num']
          filter_size = conv_param['filter_size']
          filter_pad = conv_param['pad']
          filter_stride = conv_param['stride']
          input_size = input_dim[1]
          conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
          pool_output_size = int(filter_num * (conv_output_size/2)*(conv_output_size/2))

          self.params = {}
          self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
          self.params['b1'] = np.zeros(filter_num)
          self.params['W2'] = weight_init_std * np.random.rand(pool_output_size, hidden_size)
          self.params['b2'] = np.zeros(hidden_size)
          self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
          self.params['b3'] = np.zeros(output_size)

          self.layers = OrderedDict()
          self.layers['Conv1'] = Convolution(self.params['W1'], 
                                             self.params['b1'], 
                                             conv_param['stride'], 
                                             conv_param['pad'])
          self.layers['Relu1'] = Relu()
          self.layers['Pool1'] = Pooling(pool_h = 2, pool_w = 2, stride = 2)
          self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
          self.layers['Relu1'] = Relu()
          self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
          self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
              x = layer.forward(x)

        return x

    def loss(self, x, t):
         y = self.predict(x)
         return self.lastLayer.forward(y, t)
        
    def gradient(self, x, t):
         # forward
         self.loss(x, t)

         # backward
         dout = 1
         dout = self.lastLayer.backward(dout)
         layers = list(self.layers.values())
         layers.reverse()
         for layer in layers:
              dout = layer.backward(dout)

         # 設定
         grads = {}
         grads['W1'] = self.layers['Conv1'].dW
         grads['b1'] = self.layers['Conv1'].db
         grads['W2'] = self.layers['Affine1'].dW
         grads['b2'] = self.layers['Affine1'].db
         grads['W3'] = self.layers['Affine2'].dW
         grads['b3'] = self.layers['Affine2'].db

         return grads
    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

In [4]:
# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 処理に時間のかかる場合はデータを削減 
x_train, t_train = x_train[:5000], t_train[:5000]
x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()


  0%|          | 0/1000 [00:00<?, ?it/s]

train loss:2.2968923965500205


  0%|          | 1/1000 [00:02<49:52,  3.00s/it]

=== epoch:1, train acc:0.1, test acc:0.094 ===


  0%|          | 2/1000 [00:03<24:35,  1.48s/it]

...

 50%|█████     | 500/1000 [03:44<03:14,  2.57it/s]

train loss:0.04649792529270319


 50%|█████     | 500/1000 [03:44<03:44,  2.22it/s]

train loss:0.08729490893884399


KeyboardInterrupt: 

In [ ]:
# パラメータの保存
network.save_params("params.pkl")
print("Saved Network Parameters!")

# グラフの描画
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()
